In [188]:
import numpy as np
import pandas as pd
from pathlib import Path

## Define table path, stats, latest week

In [189]:
# table_path = Path('/data/schwartzao/covid-sicr/tables/20210311_realtime/')
table_path = Path('/Users/schwartzao/Desktop/workspace/covid-sicr/tables/20210311_realtime/')
stats = ['R0', 'Rt', 'car', 'ifr', 'ir', 'ar']
last_week = '58'

In [190]:
df = pd.read_csv(table_path / 'fit_table_raw.csv', index_col=['model', 'roi', 'quantile'])
df = df[~df.index.duplicated(keep='last')]
df.columns.name = 'param'
df = df.stack('param').unstack(['roi', 'quantile', 'param']).T

In [191]:
last_week_stats = [x + f' (week {last_week})' for x in stats]
ll_waic_loo = ['ll_', 'waic', 'loo']

In [192]:
rois = df.index.get_level_values('roi').unique()

dfs = []
for roi in rois:
    df.loc[(roi, 'mean', 'll_')] *= -2 # first normalize ll
    ll_waic_loo_stats = df.loc[(roi, 'mean', ll_waic_loo)]
    other_stats = df.loc[(roi, '0.5', last_week_stats)]
    dfs.append(ll_waic_loo_stats)
    dfs.append(other_stats)

In [193]:
df_result = pd.concat(dfs)

In [194]:
# report minimum values for ll, waic, loo
columns = df_result.columns
df_result = df_result.assign(minimum = df_result[columns].min(axis=1), minimum_column=df_result[columns].idxmin(axis=1))

In [195]:
for roi in rois: # remove lowest value model values for non loo/waic/ll rows
    df_result.loc[(roi, 'mean', last_week_stats), 'minimum_column'] = ''
    df_result.loc[(roi, '0.5', last_week_stats), 'minimum_column'] = ''

In [196]:
df_result.to_csv(table_path / 'model_contributions_and_median_stats.csv')